<a href="https://colab.research.google.com/github/donbcolab/composable_vlms/blob/main/notebooks/finetuned_object_detection_using_adapters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q einops flash_attn timm peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 55.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 65.5 MB/s eta 0:00:00


In [2]:
import requests
import torch

from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

## Start with a Light Weight Vision Model - with Fine tuning capabilities

In [ ]:
base_vision_model_id = "microsoft/Florence-2-large-ft"

Set it for Object Detection mode

In [ ]:
prompt = "<OD>"

## Add a Blood Cell Image Adapter

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

blood_cell_adapter = "dwb2023/florence2-large-bccd-base-ft"

model = AutoModelForCausalLM.from_pretrained(base_vision_model_id, torch_dtype=torch_dtype, trust_remote_code=True).to(device)
model.load_adapter(blood_cell_adapter)
processor = AutoProcessor.from_pretrained(base_vision_model_id, trust_remote_code=True)

In [ ]:
url = "https://huggingface.co/spaces/dwb2023/omniscience/resolve/main/examples/BloodImage_00099_jpg.rf.0a65e56401cdd71253e7bc04917c3558.jpg"

image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)

## Switch to the Liver Disease Image Adapter

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

liver_disease_adapter = "dwb2023/florence2-large-liver-disease-ft"

model = AutoModelForCausalLM.from_pretrained(base_vision_model_id, torch_dtype=torch_dtype, trust_remote_code=True).to(device)
model.load_adapter(liver_disease_adapter)
processor = AutoProcessor.from_pretrained(base_vision_model_id, trust_remote_code=True)

In [ ]:
url = "https://huggingface.co/spaces/dwb2023/omniscience/resolve/main/examples/15_242_212_25_25_jpg.rf.f6bbadf4260dd2c1f5b4ace1b09b0a1b.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)